#Flight Delays Data
In this section we’ll walk through a few examples of queries on the Airline On-Time
Performance and Causes of Flight Delays data set, which contains data on US flights
including date, delay, distance, origin, and destination. It’s available as a CSV file with
over a million records.
We will use SQL in your Spark applications via the spark.sql programmatic interface. Similar to the DataFrame API in its declarative
flavor, this interface allows you to query structured data in your Spark
applications.

In [2]:
from pyspark.sql import SparkSession
# Create a SparkSession
spark = (SparkSession
         .builder
         .appName("SparkSQLExampleApp")
         .getOrCreate())

In [3]:
# Path to data set
csv_file = "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

# Read and create a temporary view
# Infer schema (note that for larger files you
# may want to specify the schema)
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

df.createOrReplaceTempView("us_delay_flights_tbl")

In [4]:
df.show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1011245| 6| 602| ABE| ATL|
1020600| -8| 369| ABE| DTW|
1021245| -2| 602| ABE| ATL|
1020605| -4| 602| ABE| ATL|
1031245| -4| 602| ABE| ATL|
1030605| 0| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1051245| 88| 602| ABE| ATL|
1050605| 9| 602| ABE| ATL|
1061215| -6| 602| ABE| ATL|
1061725| 69| 602| ABE| ATL|
1061230| 0| 369| ABE| DTW|
1060625| -3| 602| ABE| ATL|
1070600| 0| 369| ABE| DTW|
1071725| 0| 602| ABE| ATL|
1071230| 0| 369| ABE| DTW|
1070625| 0| 602| ABE| ATL|
1071219| 0| 569| ABE| ORD|
1080600| 0| 369| ABE| DTW|
+-------+-----+--------+------+-----------+
only showing top 20 rows

Now that we have a temporary view, we can issue SQL queries using Spark SQL.

#Query 1:
Find out all flights whose distance between origin and destination is greater than 1000

In [7]:
(spark.sql("""SELECT distance, origin, destination 
              FROM us_delay_flights_tbl 
              WHERE distance > 1000 
              ORDER BY distance DESC""")
.show(10, truncate=False))

+--------+------+-----------+
distance|origin|destination|
+--------+------+-----------+
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
+--------+------+-----------+
only showing top 10 rows

In [8]:
"""Equivalent in DataFrame"""

from pyspark.sql.types import *
from pyspark.sql.functions import *

(df.select("distance", "origin", "destination")
 .where(col("distance") > 1000)
 .orderBy(desc("distance"))
 .show(10, truncate=False))

+--------+------+-----------+
distance|origin|destination|
+--------+------+-----------+
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
+--------+------+-----------+
only showing top 10 rows

#Query 2:
Find out all flights with 2 hour delays between San Francisco and Chicago

In [10]:
(spark.sql("""SELECT date, delay, origin, destination 
              FROM us_delay_flights_tbl 
              WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
              ORDER BY delay DESC""")
.show(10, truncate=False))

+-------+-----+------+-----------+
date |delay|origin|destination|
+-------+-----+------+-----------+
2190925|1638 |SFO |ORD |
1031755|396 |SFO |ORD |
1022330|326 |SFO |ORD |
1051205|320 |SFO |ORD |
1190925|297 |SFO |ORD |
2171115|296 |SFO |ORD |
1071040|279 |SFO |ORD |
1051550|274 |SFO |ORD |
3120730|266 |SFO |ORD |
1261104|258 |SFO |ORD |
+-------+-----+------+-----------+
only showing top 10 rows

#Query 3:
A more complicated query in SQL, let's label all US flights originating from airports with high, medium, low, no delays, regardless of destinations.

In [12]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN  'Long Delays '
                  WHEN delay > 60 AND delay < 120 THEN  'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN   'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'No Delays'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10, truncate=False)

+-----+------+-----------+-------------+
delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
333 |ABE |ATL |Long Delays |
305 |ABE |ATL |Long Delays |
275 |ABE |ATL |Long Delays |
257 |ABE |ATL |Long Delays |
247 |ABE |DTW |Long Delays |
247 |ABE |ATL |Long Delays |
219 |ABE |ORD |Long Delays |
211 |ABE |ATL |Long Delays |
197 |ABE |DTW |Long Delays |
192 |ABE |ORD |Long Delays |
+-----+------+-----------+-------------+
only showing top 10 rows

## DataFrames and Spark SQL Common Relational Operators

The power of Spark SQL is that it contains many DataFrame Operations (also known as Untyped Dataset Operations). 

For the full list, refer to [Spark SQL, Built-in Functions](https://spark.apache.org/docs/latest/api/sql/index.html).

In the next section, we will focus on the following common relational operators:
* Unions and Joins
* Windowing
* Modifications

In [14]:
airports_path = "/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"

airports = spark.read.options(header='true', inferSchema='true', sep='\t').csv(airports_path)
airports.createOrReplaceTempView("airports_na")

delays = (df
          .withColumn("delay", expr("CAST(delay as INT) as delay"))
          .withColumn("distance", expr("CAST(distance as INT) as distance")))

delays.createOrReplaceTempView("departureDelays")

In [15]:
delays.show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1011245| 6| 602| ABE| ATL|
1020600| -8| 369| ABE| DTW|
1021245| -2| 602| ABE| ATL|
1020605| -4| 602| ABE| ATL|
1031245| -4| 602| ABE| ATL|
1030605| 0| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1051245| 88| 602| ABE| ATL|
1050605| 9| 602| ABE| ATL|
1061215| -6| 602| ABE| ATL|
1061725| 69| 602| ABE| ATL|
1061230| 0| 369| ABE| DTW|
1060625| -3| 602| ABE| ATL|
1070600| 0| 369| ABE| DTW|
1071725| 0| 602| ABE| ATL|
1071230| 0| 369| ABE| DTW|
1070625| 0| 602| ABE| ATL|
1071219| 0| 569| ABE| ORD|
1080600| 0| 369| ABE| DTW|
+-------+-----+--------+------+-----------+
only showing top 20 rows

Create temporary small table

In [17]:
foo = delays.filter(expr("""
            origin == 'ABE' AND 
            destination == 'ATL' AND 
            date like '104%' AND 
            delay > 0"""))

foo.createOrReplaceTempView("foo")

In [18]:
foo.show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
+-------+-----+--------+------+-----------+

In [19]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
 City|State|Country|IATA|
+-----------+-----+-------+----+
 Abbotsford| BC| Canada| YXX|
 Aberdeen| SD| USA| ABR|
 Abilene| TX| USA| ABI|
 Akron| OH| USA| CAK|
 Alamosa| CO| USA| ALS|
 Albany| GA| USA| ABY|
 Albany| NY| USA| ALB|
Albuquerque| NM| USA| ABQ|
 Alexandria| LA| USA| AEX|
 Allentown| PA| USA| ABE|
+-----------+-----+-------+----+

In [20]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1011245| 6| 602| ABE| ATL|
1020600| -8| 369| ABE| DTW|
1021245| -2| 602| ABE| ATL|
1020605| -4| 602| ABE| ATL|
1031245| -4| 602| ABE| ATL|
1030605| 0| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1051245| 88| 602| ABE| ATL|
1050605| 9| 602| ABE| ATL|
+-------+-----+--------+------+-----------+

In [21]:
spark.sql("SELECT * FROM foo LIMIT 10").show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
+-------+-----+--------+------+-----------+

* Union two tables

In [23]:
bar = delays.union(foo)
bar.createOrReplaceTempView("bar")
bar.filter(expr("origin == 'ABE' AND destination == 'ATL' AND date LIKE '104%' AND delay > 0")).show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
+-------+-----+--------+------+-----------+

In [24]:
spark.sql("""
SELECT * 
FROM bar 
WHERE origin = 'ABE' 
   AND destination = 'ATL' 
   AND date LIKE '104%'
   AND delay > 0
""").show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
+-------+-----+--------+------+-----------+

In [25]:
bar.show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1011245| 6| 602| ABE| ATL|
1020600| -8| 369| ABE| DTW|
1021245| -2| 602| ABE| ATL|
1020605| -4| 602| ABE| ATL|
1031245| -4| 602| ABE| ATL|
1030605| 0| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1051245| 88| 602| ABE| ATL|
1050605| 9| 602| ABE| ATL|
1061215| -6| 602| ABE| ATL|
1061725| 69| 602| ABE| ATL|
1061230| 0| 369| ABE| DTW|
1060625| -3| 602| ABE| ATL|
1070600| 0| 369| ABE| DTW|
1071725| 0| 602| ABE| ATL|
1071230| 0| 369| ABE| DTW|
1070625| 0| 602| ABE| ATL|
1071219| 0| 569| ABE| ORD|
1080600| 0| 369| ABE| DTW|
+-------+-----+--------+------+-----------+
only showing top 20 rows

* Join two tables

In [27]:
# Join Departure Delays data (foo) with flight info
foo.join(
  airports, 
  airports.IATA == foo.origin
).select("City", "State", "date", "delay", "distance", "destination").show()

+---------+-----+-------+-----+--------+-----------+
 City|State| date|delay|distance|destination|
+---------+-----+-------+-----+--------+-----------+
Allentown| PA|1041243| 10| 602| ATL|
Allentown| PA|1040605| 28| 602| ATL|
+---------+-----+-------+-----+--------+-----------+

In [28]:
spark.sql("""
SELECT a.City, a.State, f.date, f.delay, f.distance, f.destination 
  FROM foo f
  JOIN airports_na a
    ON a.IATA = f.origin
""").show()

+---------+-----+-------+-----+--------+-----------+
 City|State| date|delay|distance|destination|
+---------+-----+-------+-----+--------+-----------+
Allentown| PA|1041243| 10| 602| ATL|
Allentown| PA|1040605| 28| 602| ATL|
+---------+-----+-------+-----+--------+-----------+

## Windowing Functions

Great reference: [Introduction Windowing Functions in Spark SQL](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html)

> At its core, a window function calculates a return value for every input row of a table based on a group of rows, called the Frame. Every input row can have a unique frame associated with it. This characteristic of window functions makes them more powerful than other functions and allows users to express various data processing tasks that are hard (if not impossible) to be expressed without window functions in a concise way.

In [30]:
spark.sql("DROP TABLE IF EXISTS departureDelaysWindow")
spark.sql("""
CREATE TABLE departureDelaysWindow AS
SELECT origin, destination, sum(delay) as TotalDelays 
  FROM departureDelays 
 WHERE origin IN ('SEA', 'SFO', 'JFK') 
   AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL') 
 GROUP BY origin, destination
""")

spark.sql("""SELECT * FROM departureDelaysWindow""").show()

+------+-----------+-----------+
origin|destination|TotalDelays|
+------+-----------+-----------+
 JFK| ORD| 5608|
 SEA| LAX| 9359|
 JFK| SFO| 35619|
 SFO| ORD| 27412|
 JFK| DEN| 4315|
 SFO| DEN| 18688|
 SFO| SEA| 17080|
 SEA| SFO| 22293|
 JFK| ATL| 12141|
 SFO| ATL| 5091|
 SEA| DEN| 13645|
 SEA| ATL| 4535|
 JFK| SEA| 7856|
 SFO| LAX| 40798|
 SEA| JFK| 4667|
 SEA| ORD| 10041|
 JFK| LAX| 35755|
 SFO| JFK| 24100|
+------+-----------+-----------+

What are the top three total delays destinations by origin city of SEA, SFO, and JFK?

In [32]:
spark.sql("""
SELECT origin, destination, sum(TotalDelays) as TotalDelays
 FROM departureDelaysWindow
WHERE origin = 'SEA'
GROUP BY origin, destination
ORDER BY sum(TotalDelays) DESC
LIMIT 3
""").show()

+------+-----------+-----------+
origin|destination|TotalDelays|
+------+-----------+-----------+
 SEA| SFO| 22293|
 SEA| DEN| 13645|
 SEA| ORD| 10041|
+------+-----------+-----------+

What if for each of these origin airports you wanted to find the three destinations that
experienced the most delays? You could achieve this by running three different queries
for each origin and then unioning the results together.

But a better approach would be to use a window function like dense_rank() to perform
the following calculation:

In [34]:
spark.sql("""
SELECT origin, destination, TotalDelays, rank 
  FROM ( 
     SELECT origin, destination, TotalDelays, dense_rank() 
       OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank 
       FROM departureDelaysWindow
  ) t 
 WHERE rank <= 3
""").show()

+------+-----------+-----------+----+
origin|destination|TotalDelays|rank|
+------+-----------+-----------+----+
 SEA| SFO| 22293| 1|
 SEA| DEN| 13645| 2|
 SEA| ORD| 10041| 3|
 SFO| LAX| 40798| 1|
 SFO| ORD| 27412| 2|
 SFO| JFK| 24100| 3|
 JFK| LAX| 35755| 1|
 JFK| SFO| 35619| 2|
 JFK| ATL| 12141| 3|
+------+-----------+-----------+----+

## Modifications

Another common DataFrame operation is to perform modifications to the DataFrame. Recall that the underlying RDDs are immutable (i.e. they do not change) to ensure there is data lineage for Spark operations. Hence while DataFrames themselves are immutable, you can modify them through operations that create a new, different DataFrame with different columns, for example.

In [36]:
foo.show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
+-------+-----+--------+------+-----------+

* adding new columns

In [38]:
foo2 = foo.withColumn("status", expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END"))
foo2.show()

+-------+-----+--------+------+-----------+-------+
 date|delay|distance|origin|destination| status|
+-------+-----+--------+------+-----------+-------+
1041243| 10| 602| ABE| ATL|On-time|
1040605| 28| 602| ABE| ATL|Delayed|
+-------+-----+--------+------+-----------+-------+

* dropping columns

In [40]:
foo3 = foo2.drop("delay")
foo3.show()

+-------+--------+------+-----------+-------+
 date|distance|origin|destination| status|
+-------+--------+------+-----------+-------+
1041243| 602| ABE| ATL|On-time|
1040605| 602| ABE| ATL|Delayed|
+-------+--------+------+-----------+-------+

* Renaming columns

In [42]:
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

+-------+--------+------+-----------+-------------+
 date|distance|origin|destination|flight_status|
+-------+--------+------+-----------+-------------+
1041243| 602| ABE| ATL| On-time|
1040605| 602| ABE| ATL| Delayed|
+-------+--------+------+-----------+-------------+

* Pivoting

In [44]:
spark.sql("""SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay FROM departureDelays WHERE origin = 'SEA'""").show(10)

+-----------+-----+-----+
destination|month|delay|
+-----------+-----+-----+
 ORD| 10| 92|
 JFK| 10| -7|
 DFW| 10| -5|
 MIA| 10| -3|
 DFW| 10| -3|
 DFW| 10| 1|
 ORD| 10| -10|
 DFW| 10| -6|
 DFW| 10| -2|
 ORD| 10| -3|
+-----------+-----+-----+
only showing top 10 rows

In [45]:
spark.sql("""
SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay 
  FROM departureDelays WHERE origin = 'SEA'
) 
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) as AvgDelay, MAX(delay) as MaxDelay
  FOR month IN (10 OCT, 11 NOV, 12 DEC)
)
ORDER BY destination
""").show()

+-----------+------------+------------+------------+------------+------------+------------+
destination|OCT_AvgDelay|OCT_MaxDelay|NOV_AvgDelay|NOV_MaxDelay|DEC_AvgDelay|DEC_MaxDelay|
+-----------+------------+------------+------------+------------+------------+------------+
 ABQ| 12.00| 27| 8.25| 21| 41.00| 316|
 ANC| 9.81| 149| 3.58| 106| -0.09| 52|
 ATL| 18.74| 397| 7.89| 201| 7.04| 206|
 AUS| 11.44| 50| 1.10| 31| -3.10| 3|
 BOS| 16.19| 110| 7.81| 110| 1.22| 110|
 BUR| -4.67| 4| -1.90| 54| -0.87| 56|
 CLE| 16.00| 27| null| null| null| null|
 CLT| 4.59| 41| 2.47| 33| -1.43| 12|
 COS| 12.56| 82| 3.90| 42| 2.60| 53|
 CVG| -0.50| 4| null| null| null| null|
 DCA| 0.50| 50| -5.26| 2| 1.25| 45|
 DEN| 26.99| 425| 7.69| 120| 4.69| 137|
 DFW| 12.00| 116| 6.08| 132| 7.08| 247|
 DTW| 22.09| 107| 2.84| 28| 2.72| 25|
 EWR| 13.61| 236| 9.76| 138| 1.94| 76|
 FAI| 9.27| 160| -0.54| 28| -2.31| 17|
 FAT| 5.29| 119| 2.40| 88| -3.80| 18|
 FLL| 7.22| 46| 1.00| 33| 2.30| 54|
 GEG| 1.37| 48| 1.80| 63| 2.83| 44|
 HDN| -3.00| -1| -5.33| 5| 7.00| 27|
+-----------+------------+------------+------------+------------+------------+------------+
only showing top 20 rows